## Create trade history for issueplus or minus a given number of maturity-years. 

Last updated by Developer 06-18-2025

This notebook creates views with histories of trades for CUSIPs in the same issue with similar coupons and similar maturties.  The hope is that these trades will help accuracy for illiquid CUSIPs where there is a trade for a similar CUSIP. 

In [ ]:
import os
from google.cloud import bigquery


os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/user/base/ficc/creds.json'
bq_client = bigquery.Client()

project = 'eng-reactor-287421'

In [ ]:
def mkview(dataset, name, sql):
    db = f'{project}.{dataset}.'
    print(sql)
    name = db + name
    bq_client.delete_table(name, not_found_ok=True) 
    view = bigquery.Table(name)
    view.view_query = sql
    view = bq_client.create_table(view)
    return name

In [ ]:
def sqltodf(sql, limit=''):
    if limit != '': 
        limit = f' ORDER BY RAND() LIMIT {limit}'
    bqr = bq_client.query(sql + limit).result()
    return bqr.to_dataframe()

In [ ]:
trans_with_buckets = mkview('auxiliary_views_v2', 'trans_with_buckets',
  f'''SELECT *, 
  CONCAT(CAST(issue_key AS STRING), '_', CAST(maturity_bucket AS STRING), '_', CAST(coupon_bucket AS STRING)) AS issue_mat_coup_key 
  from (SELECT 
  CAST(TRUNC(DATE_DIFF(maturity_date,trade_date, day)/360) / 5 AS INT64) AS maturity_bucket,
  CAST(TRUNC(DATE_DIFF(calc_date,trade_date, day)/360) / 5 AS INT64) AS calc_date_bucket,
  CASE
    WHEN coupon = 0 THEN -1 --arbitrary value
  ELSE
  CAST( TRUNC(coupon/1) AS int64)
END
  AS coupon_bucket,
  *
    FROM eng-reactor-287421.auxiliary_views_v2.msrb_trans)''')

print(trans_with_buckets)                             

In [ ]:
trade_history_same_issue_5_yr_mat_bucket_1_coupon = mkview('auxiliary_views_v2', 'trade_history_same_issue_5_yr_mat_bucket_1_coupon',
f'''
 SELECT
        latest.rtrs_control_number,
        latest.publish_datetime,
        latest.sequence_number as seq_num,
        ARRAY_AGG(STRUCT (past.msrb_valid_from_date, past.msrb_valid_to_date,past.rtrs_control_number, past.trade_datetime, past.publish_datetime,
                         past.yield, past.dollar_price, past.par_traded, past.trade_type,
                   datetime_diff(latest.trade_datetime,past.trade_datetime, second) as seconds_ago, past.is_non_transaction_based_compensation,
                        past.is_lop_or_takedown, past.brokers_broker, past.is_alternative_trading_system, past.is_weighted_average_price, past.settlement_date,
                        past.calc_date,past.calc_day_cat,past.coupon, past.maturity_date, past.next_call_date, past.par_call_date,past.refund_date, latest.rtrs_control_number as current_rtrs_control_number, latest.par_traded - past.par_traded as par_traded_diff,
      concat(past.trade_type,latest.trade_type) as trade_type_past_latest)
                ORDER BY past.trade_datetime DESC, past.publish_datetime DESC, past.sequence_number   LIMIT 10
              ) AS recent_5_year_mat
    FROM {trans_with_buckets} latest
    LEFT JOIN {trans_with_buckets}  past
        ON
        latest.issue_key = past.issue_key
        AND latest.cusip <> past.cusip 
        AND past.msrb_valid_to_date > current_datetime('America/New_York') 
        AND (past.transaction_type <> 'C' or past.transaction_type is null) 
        AND latest.trade_datetime > past.publish_datetime
        AND latest.maturity_bucket = past.maturity_bucket
        AND latest.coupon_bucket = past.coupon_bucket
    GROUP BY latest.rtrs_control_number, latest.publish_datetime, latest.sequence_number
''')

print(trade_history_same_issue_5_yr_mat_bucket_1_coupon)

In [ ]:
trade_history_ref_data_same_issue_plus_5_yr_mat_bucket_1_coupon = mkview('auxiliary_views_v2', 'trade_history_ref_data_same_issue_5_yr_mat_bucket_1_coupon',
f'''
 SELECT
      thrd.*,
      latest.* EXCEPT (publish_datetime,
        rtrs_control_number, seq_num)
    FROM eng-reactor-287421.primary_views_v2.trade_history_with_reference_data thrd
    LEFT JOIN
      {trade_history_same_issue_5_yr_mat_bucket_1_coupon} latest
    ON
      thrd.rtrs_control_number = latest.rtrs_control_number
      AND thrd.publish_datetime = latest.publish_datetime
      --AND thrd.sequence_number = latest.seq_num

''')

print(trade_history_ref_data_same_issue_plus_5_yr_mat_bucket_1_coupon)

In [ ]:
sqltodf(f'''CREATE OR REPLACE TABLE
  auxiliary_views_v2.trade_history_same_issue_5_yr_mat_bucket_1_materialized AS
SELECT
*
FROM
eng-reactor-287421.auxiliary_views_v2.trade_history_ref_data_same_issue_5_yr_mat_bucket_1_coupon''')